In [1]:
def get_input(fname='test.txt'):
    input = []
    with open(fname) as f:
        for l in f.readlines():
            line = l.rstrip()
            direction, length, hex_code = line.split(' ')
            input.append([direction, int(length), hex_code[2:-1]])
    return input

In [2]:
test_input = get_input('test.txt')
my_input = get_input('input.txt')

In [3]:
test_input

[['R', 6, '70c710'],
 ['D', 5, '0dc571'],
 ['L', 2, '5713f0'],
 ['D', 2, 'd2c081'],
 ['R', 2, '59c680'],
 ['D', 2, '411b91'],
 ['L', 5, '8ceee2'],
 ['U', 2, 'caa173'],
 ['L', 1, '1b58a2'],
 ['U', 2, 'caa171'],
 ['R', 2, '7807d2'],
 ['U', 3, 'a77fa3'],
 ['L', 2, '015232'],
 ['U', 2, '7a21e3']]

In [4]:
U, D, L, R = (-1, 0), (1, 0), (0, -1), (0, 1)
DIRECTIONS = dict(U=U, D=D, L=L, R=R)

In [5]:
# inside point with clockwise direction
INSIDE = {
    L: U,
    U: R,
    R: D,
    D: L
}

In [6]:
def area(input):
    DUG = '#'
    current = (0, 0)
    grid = {0: {0: DUG}}
    to_fill = []
    for direction, length, _ in input:
        delta = DIRECTIONS[direction]
        for _ in range(length):
            current = (current[0] + delta[0], current[1] + delta[1])
            if current[0] not in grid:
                grid[current[0]] = {}
            grid[current[0]][current[1]] = DUG
            to_fill.append((current[0] + INSIDE[delta][0], current[1] + INSIDE[delta][1]))
    # fill the area:
    while to_fill:
        p = to_fill.pop()
        if grid.get(p[0]).get(p[1]) == DUG:
            continue
        if p[0] not in grid:
            grid[p[0]] = {}
        grid[p[0]][p[1]] = DUG
        for delta in DIRECTIONS.values():
            to_fill.append((p[0] + delta[0], p[1] + delta[1]))
    
    return sum(v == DUG for g in grid.values() for v in g.values())        

In [7]:
area(test_input)

62

In [8]:
area(my_input)

49061

In [9]:
from collections import deque
import heapq

In [10]:
def area_opt(input):
    horizontal_points = { 0 }
    vertical_segments = []
    current = (0, 0)
    for direction, length, _ in input:
        delta = DIRECTIONS[direction]
        nxt = [current[0] + length * delta[0], current[1] + length * delta[1]]
        if direction in ('U', 'D'):
            vertical_segments.append(sorted([current, nxt]))
        horizontal_points.add(nxt[0])
        current = nxt
    horizontal_points = sorted(horizontal_points)
    vertical_segments.sort(key=lambda s: s[0][1])
    prev = horizontal_points[0]
    rectangles = []
    for crt in horizontal_points:
        if crt == prev:
            continue
        prev_v = None
        for i in range(len(vertical_segments) - 1, -1, -1):
            v = vertical_segments[i]
            if v[0][0] != prev:
                continue
            if prev_v is not None:
                rectangles.append(((prev, v[0][1]), (crt, prev_v[0][1])))
                prev_v = None
            else:
                prev_v = v
            if v[1][0] > crt:
                v[0][0] = crt
            else:
                del vertical_segments[i]
        prev = crt
    # print('#', rectangles)

    rectangles.sort()
    area = 0
    for r in rectangles:
        area += (r[1][0] - r[0][0] + 1) * (r[1][1] - r[0][1] + 1)

    # vertical overlaps were counted twice
    for i in range(len(rectangles) - 1):
        j = i + 1
        for j in range(1 + i, len(rectangles)):
            if rectangles[i][1][0] == rectangles[j][0][0]:
                lower = (rectangles[i][0][1], rectangles[i][1][1])
                upper = (rectangles[j][0][1], rectangles[j][1][1])
                overlap = (max(lower[0], upper[0]), min(lower[1], upper[1]))
                if overlap[1] > overlap[0]:
                    area -= overlap[1] - overlap[0] + 1
            
    return area

In [11]:
area_opt(test_input) # 62

62

In [12]:
area(my_input)

49061

In [13]:
area_opt(my_input)

49061

In [14]:
test_input

[['R', 6, '70c710'],
 ['D', 5, '0dc571'],
 ['L', 2, '5713f0'],
 ['D', 2, 'd2c081'],
 ['R', 2, '59c680'],
 ['D', 2, '411b91'],
 ['L', 5, '8ceee2'],
 ['U', 2, 'caa173'],
 ['L', 1, '1b58a2'],
 ['U', 2, 'caa171'],
 ['R', 2, '7807d2'],
 ['U', 3, 'a77fa3'],
 ['L', 2, '015232'],
 ['U', 2, '7a21e3']]

In [15]:
def fix_input(input):
    d = 'RDLU'
    fixed = []
    for _, _, to_fix in input:
        fixed.append([d[int(to_fix[-1])], int(to_fix[:-1], 16), to_fix])
    return fixed

In [16]:
fixed_test_input = fix_input(test_input)

In [17]:
fixed_test_input

[['R', 461937, '70c710'],
 ['D', 56407, '0dc571'],
 ['R', 356671, '5713f0'],
 ['D', 863240, 'd2c081'],
 ['R', 367720, '59c680'],
 ['D', 266681, '411b91'],
 ['L', 577262, '8ceee2'],
 ['U', 829975, 'caa173'],
 ['L', 112010, '1b58a2'],
 ['D', 829975, 'caa171'],
 ['L', 491645, '7807d2'],
 ['U', 686074, 'a77fa3'],
 ['L', 5411, '015232'],
 ['U', 500254, '7a21e3']]

In [18]:
area_opt(fixed_test_input)

952408144115

In [19]:
fixed_my_input = fix_input(my_input)

In [20]:
area_opt(fixed_my_input)

92556825427032